In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# ler o arquivo CSV
df = pd.read_csv(f"dataset/adult.csv")
print(df.head())

   age workclass  fnlwgt     education  education.num marital.status  \
0   90         ?   77053       HS-grad              9        Widowed   
1   82   Private  132870       HS-grad              9        Widowed   
2   66         ?  186061  Some-college             10        Widowed   
3   54   Private  140359       7th-8th              4       Divorced   
4   41   Private  264663  Some-college             10      Separated   

          occupation   relationship   race     sex  capital.gain  \
0                  ?  Not-in-family  White  Female             0   
1    Exec-managerial  Not-in-family  White  Female             0   
2                  ?      Unmarried  Black  Female             0   
3  Machine-op-inspct      Unmarried  White  Female             0   
4     Prof-specialty      Own-child  White  Female             0   

   capital.loss  hours.per.week native.country income  
0          4356              40  United-States  <=50K  
1          4356              18  United-States

DATA CLEANING

In [4]:
# limpar os dados
df = df.replace("?", np.nan)
# contar valores nulos em cada coluna
print(df.isnull().sum())

age                  0
workclass         1836
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     583
income               0
dtype: int64


In [5]:
# Colunas com missing
cols_missing = ["workclass", "occupation", "native.country"]

# Mostrar a moda de cada uma
for col in cols_missing:
    print(f"Moda de {col}: {df[col].mode()[0]}")

Moda de workclass: Private
Moda de occupation: Prof-specialty
Moda de native.country: United-States


In [6]:
# Preencher NaN com a moda
for col in cols_missing:
    df[col] = df[col].fillna(df[col].mode()[0])

print("\nValores nulos depois do preenchimento:")
print(df[cols_missing].isnull().sum())


Valores nulos depois do preenchimento:
workclass         0
occupation        0
native.country    0
dtype: int64


In [7]:
# Mostrar distribuição (top 5 categorias) após preenchimento
for col in cols_missing:
    print(f"\nDistribuição em {col}:")
    print(df[col].value_counts().head())


Distribuição em workclass:
workclass
Private             24532
Self-emp-not-inc     2541
Local-gov            2093
State-gov            1298
Self-emp-inc         1116
Name: count, dtype: int64

Distribuição em occupation:
occupation
Prof-specialty     5983
Craft-repair       4099
Exec-managerial    4066
Adm-clerical       3770
Sales              3650
Name: count, dtype: int64

Distribuição em native.country:
native.country
United-States    29753
Mexico             643
Philippines        198
Germany            137
Canada             121
Name: count, dtype: int64


In [8]:
# salvar o dataset limpo para csv
df.to_csv("dataset/adult_clean.csv", index=False)

NORMALIZATIION

### 1. **Z-Score (StandardScaler)**

$$
x' = \frac{x - \mu}{\sigma}
$$

* Resultado: média = 0, desvio padrão = 1.
* Bom quando os dados têm distribuição “espalhada” (como `age`, `hours.per.week`).
* Problema: **sensível a outliers** (ex.: `capital.gain` que tem valores gigantes).

---

### 2. **Min-Max (0–1)**

$$
x' = \frac{x - x_{min}}{x_{max} - x_{min}}
$$

* Resultado: todos os valores ficam entre 0 e 1.
* Preserva proporções.
* Problema: se tiver um outlier extremo, todos os outros valores ficam espremidos.

---

## 🔹 No Adult Income:

* **age**: entre \~17 e 90 → ambos funcionam.
* **hours.per.week**: entre 1 e 99 → ambos funcionam.
* **education.num**: entre 1 e 16 → ambos funcionam.
* **capital.gain / capital.loss**: muito desbalanceados (a maioria é 0, poucos muito altos) → aqui **Z-score ajuda mais**.
* **fnlwgt**: escala gigante (20k – 1M) e pouca relevância → você pode até **remover** essa feature.

ESCOLHIDO: **Z-Score (StandardScaler)** para todas as numéricas.
É o padrão para redes neurais, deixa tudo em média 0 e desvio 1, e facilita a convergência.
